# Example of using MLSO data API

## Raw MLSO data API using requests to make web queries

Using the raw MLSO data API requires the use of some libraries such as `requests` to query the web server and `json` to parse the results. We also import some libraries for nicer formatting of the results.

In [1]:
api_baseurl = "http://api.mlso.ucar.edu"
api_version = "v1"

import json
import requests

import pandas as pd
from IPython.display import HTML, display

import os

We define a simple routine to display JSON results.

In [2]:
def print_json(s):
    print(json.dumps(s, indent=4))

Now we are ready to begin making requests from the API. To start, let's find some basic information about the API server.

In [3]:
about = requests.get(f"{api_baseurl}/{api_version}/about").json()
print_json(about)

{
    "documentation": "https://mlso-api-client.readthedocs.io/en/latest/",
    "homepage": "https://www2.hao.ucar.edu/mlso",
    "support": "mlso_data_requests@ucar.edu",
    "version": "0.3.1"
}


Next, let's find the instruments whose data is available. More legacy instruments will be added in the future.

In [4]:
instruments = requests.get(f"{api_baseurl}/{api_version}/instruments").json()
print_json(instruments)

[
    "ucomp",
    "kcor"
]


Let's collect some information about each of these instruments, such as full name, the identifier we are to refer to them by in the API, and the dates that data are available for them.

In [5]:
info = []
for i in instruments:
    instrument_info = requests.get(f"{api_baseurl}/{api_version}/instruments/{i}").json()
    info.append({
        "Instrument": instrument_info["name"],
        "ID": i,
        "Start Date": instrument_info["dates"]["start-date"],
        "End Date": instrument_info["dates"]["end-date"]
    })

display(HTML(pd.DataFrame(info, columns=["Instrument", "ID", "Start Date", "End Date"]).to_html()))

,Instrument,ID,Start Date,End Date
0,Upgraded Coronal Multi-Polarimeter (UCoMP),ucomp,2021-07-15T17:31:43,2025-03-24T21:03:55
1,COSMO K-Coronagraph (KCor),kcor,2013-09-30T18:57:54,2025-03-24T21:04:20


Next, let's query for the products available for one of the instruments.

In [6]:
info = requests.get(f"{api_baseurl}/{api_version}/instruments/ucomp/products").json()
products = info["products"]
display(HTML(pd.DataFrame(products, columns=["title", "id", "description"]).to_html()))

,title,id,description
0,Level 1,l1,IQUV and backgrounds for various wavelengths
1,Level 1 intensity,intensity,intensity-only level 1
2,Level 1 mean,mean,mean of level 1 files
3,Level 1 median,median,median of level 1 files
4,Level 1 sigma,sigma,standard deviation of level 1 files
5,Level 2,l2,level 2 products
6,Level 2 average,l2average,"mean, median, standard deviation of level 2 files"
7,Density,density,density
8,Dynamics,dynamics,level 2 dynamics products
9,Polarization,polarization,level 2 polarization products


In [7]:
info = requests.get(f"{api_baseurl}/{api_version}/instruments/ucomp/products/l2?wave-region=789&start-date=2025-03-23").json()
print_json(info)
display(HTML(pd.DataFrame(info["files"], columns=["filename", "url"]).to_html()))

{
    "end-date": "2025-03-24T21:03:55",
    "files": [
        {
            "date-obs": "2025-03-23T19:03:36",
            "filename": "20250323.190336.ucomp.789.l2.fts",
            "filesize": 31501440,
            "instrument": "ucomp",
            "obs-plan": "synoptic-original-lines.cbk",
            "product": "l2",
            "url": "http://api.mlso.ucar.edu/v1/download?obsday-id=10136&client=API&instrument=ucomp&filename=20250323.190336.ucomp.789.l2.fts",
            "wave-region": "789",
            "wavelengths": 5
        },
        {
            "date-obs": "2025-03-24T20:06:52",
            "filename": "20250324.200652.ucomp.789.l2.fts",
            "filesize": 31501440,
            "instrument": "ucomp",
            "obs-plan": "synoptic-original-lines.cbk",
            "product": "l2",
            "url": "http://api.mlso.ucar.edu/v1/download?obsday-id=10137&client=API&instrument=ucomp&filename=20250324.200652.ucomp.789.l2.fts",
            "wave-region": "789",
      

,filename,url
0,20250323.190336.ucomp.789.l2.fts,http://api.mlso.ucar.edu/v1/download?obsday-id=10136&client=API&instrument=ucomp&filename=20250323.190336.ucomp.789.l2.fts
1,20250324.200652.ucomp.789.l2.fts,http://api.mlso.ucar.edu/v1/download?obsday-id=10137&client=API&instrument=ucomp&filename=20250324.200652.ucomp.789.l2.fts


Downloading a file requires first registering an email address with the HAO website. If you don't already have an account registered, go to [the registration page] to register one.

[the registration page]: https://registration.hao.ucar.edu

In [8]:
filename = info["files"][0]["filename"]
url = info["files"][0]["url"]

session = requests.Session()

# authenticate first
username = "email@example.com"
r = session.get(url = f"{api_baseurl}/authenticate?username={username}")

# then download
r = session.get(url, stream=True, cookies=session.cookies.get_dict())
with open(filename, "wb") as handle:
    for data in r.iter_content(chunk_size=1024 * 1024):
        handle.write(data)

if os.path.exists(filename):
    print(f"{filename} downloaded")

20250323.190336.ucomp.789.l2.fts downloaded


## mlso.api.client API

The `mlso.api.client` module provides convenience routines to avoid the use of low-level libraries dealing with formulating the URLs in the webservice requests, making the requests, parsing JSON responses, etc.

In [9]:
from mlso.api import client

print(f"Using client version v{client.__version__}")

Using client version v0.3.2


Instead of having to construct URLs and make the web requests, we can now use routines provides by the client API. The `about` routine now gives information about the API web server.

In [10]:
info = client.about()
print(f"Using mlsoapi v{info['version']}, see {info['documentation']} for more information.")

Using mlsoapi v0.3.1, see https://mlso-api-client.readthedocs.io/en/latest/ for more information.


The `instruments` routine combines potentially many API calls to give basic information about all the available instruments.

In [11]:
info = client.instruments()
display(HTML(pd.DataFrame(info, columns=["name", "id", "start-date", "end-date"]).to_html()))

,name,id,start-date,end-date
0,COSMO K-Coronagraph (KCor),kcor,2013-09-30T18:57:54,2025-03-24T21:04:20
1,Upgraded Coronal Multi-Polarimeter (UCoMP),ucomp,2021-07-15T17:31:43,2025-03-24T21:03:55


The `products` routine gives a list of products with basic information.

In [12]:
info = client.products("ucomp")["products"]
display(HTML(pd.DataFrame(info, columns=["title", "id", "description"]).to_html()))

,title,id,description
0,Level 1,l1,IQUV and backgrounds for various wavelengths
1,Level 1 intensity,intensity,intensity-only level 1
2,Level 1 mean,mean,mean of level 1 files
3,Level 1 median,median,median of level 1 files
4,Level 1 sigma,sigma,standard deviation of level 1 files
5,Level 2,l2,level 2 products
6,Level 2 average,l2average,"mean, median, standard deviation of level 2 files"
7,Density,density,density
8,Dynamics,dynamics,level 2 dynamics products
9,Polarization,polarization,level 2 polarization products


The `files` routine provides a list of files, with URLs to download them, that match a set of filters.

In [13]:
info = client.files("ucomp", "l2", {"wave-region": "789", "start-date": "2025-03-23"})
display(HTML(pd.DataFrame(info["files"], columns=["filename", "url"]).to_html()))

,filename,url
0,20250323.190336.ucomp.789.l2.fts,http://api.mlso.ucar.edu/v1/download?obsday-id=10136&client=python&instrument=ucomp&filename=20250323.190336.ucomp.789.l2.fts
1,20250324.200652.ucomp.789.l2.fts,http://api.mlso.ucar.edu/v1/download?obsday-id=10137&client=python&instrument=ucomp&filename=20250324.200652.ucomp.789.l2.fts


The files can be downloaded, though the `authenticate` routine must be called before starting to download files. An email address must be registered with the HAO website to download files. Use [the registration page] mentioned previously to register one.

[the registration page]: https://registration.hao.ucar.edu

In [14]:
client.authenticate("email@example.com")

output_dir = "./data"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

for file in info["files"]:
    path = client.download_file(file, output_dir)
    print(f"downloaded {file['filename']} to {path}")

downloaded 20250323.190336.ucomp.789.l2.fts to data/20250323.190336.ucomp.789.l2.fts
downloaded 20250324.200652.ucomp.789.l2.fts to data/20250324.200652.ucomp.789.l2.fts
